In [1]:
import schedule
import time
from datetime import datetime

def generate_statistics():
    import requests 
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from collections import Counter
    import os

    # Obtener los datos desde el servidor
    url = "http://conexus.dam.inspedralbes.cat:24849/users"
    response = requests.get(url)

    url2 = "http://conexus.dam.inspedralbes.cat:24849/typesUsers"
    response2 = requests.get(url2)

    # Verificar si las solicitudes fueron exitosas
    if response.status_code != 200 or response2.status_code != 200:
        raise Exception(f"Error al hacer el fetch: {response.status_code}, {response2.status_code}")

    # Parsear los datos JSON
    users = response.json()
    types_users_data = response2.json()

    # Crear un diccionario para mapear ID de tipos de usuario a nombres
    types_users_mapping = {type_user['id']: type_user['name'] for type_user in types_users_data}

    # Extraer los tipos de usuario
    types_users = [user['typesUsers_id'] for user in users]

    # Extraer las valoraciones de los usuarios
    reviews = [float(user['review']) for user in users]

    # Extraer las ciudades de los usuarios
    cities = [user['city'].split(',')[0] for user in users]

    # Cálculos de estadísticas
    types_counter = dict(Counter(types_users))
    average_review = np.mean(reviews)
    max_review = np.max(reviews)
    min_review = np.min(reviews)

    # Obtener la fecha actual para los nombres de archivo
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Crear la carpeta si no existe
    os.makedirs(f"images/statistics/users/{current_date}", exist_ok=True)

    # Gráfica de porcentaje de cada tipo de usuario (Gráfico de Pastel)
    types_labels = [types_users_mapping[type_id] for type_id in types_counter.keys()]
    plt.figure(figsize=(8, 6))
    plt.pie(types_counter.values(), labels=types_labels, autopct='%1.1f%%', colors=sns.color_palette("Set2", len(types_counter)))
    plt.title('Distribució de Tipus d\'Usuari')
    plt.axis('equal')
    plt.savefig(f"images/statistics/users/{current_date}/types_users.png")

    # Gráfica de valoración de usuarios (Boxplot)
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=reviews, color="skyblue")
    plt.title(f'Distribució de Valoracions d\'Usuaris (Mitjana: {average_review:.2f})')
    plt.xlabel('Valoració')
    plt.axvline(x=average_review, color='red', linestyle='--', label=f'Mitjana: {average_review:.2f}')
    plt.axvline(x=max_review, color='green', linestyle='--', label=f'Màxima: {max_review:.2f}')
    plt.axvline(x=min_review, color='orange', linestyle='--', label=f'Mínima: {min_review:.2f}')
    plt.legend()
    plt.savefig(f"images/statistics/users/{current_date}/reviews_distribution.png")

    # Gráfica de ciudades de los usuarios (Gráfico de barras horizontal)
    city_counter = dict(Counter(cities))
    plt.figure(figsize=(8, 6))
    sns.barplot(x=list(city_counter.values()), y=list(city_counter.keys()), palette="Set3", orient="h")
    plt.title('Usuaris per Ciutat')
    plt.xlabel('Quantitat d\'Usuaris')
    plt.ylabel('Ciutat')
    plt.tight_layout()
    plt.savefig(f"images/statistics/users/{current_date}/users_by_city.png")

    print(f"Gràfiques generades i guardades amb el nom de data actual a 'images/statistics'.")

# Programar la tarea
schedule.every().tuesday.at("10:00").do(generate_statistics)

print("Esperando para ejecutar las estadísticas cada viernes a las 23:59...")

# Mantener el script en ejecución
while True:
    schedule.run_pending()
    time.sleep(1)


Esperando para ejecutar las estadísticas cada viernes a las 23:59...


KeyError: 'typesUsers_id'